In [1]:
from mylib.model import heat_model
import mylib.integration as integration

import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column

output_notebook(hide_banner=True)

# The heat equation

We use a discretization through the method of lines (MOL) of the heat equation :

\begin{equation} 
\left\{
\begin{aligned}
&\partial_t  u(t,x) = \partial^2_{xx} u(t,x) \qquad{} t\geq t_0 \\
&u(t_0,x) = u_0(x)
\end{aligned}
\right.
\end{equation}

In a first part, we focus on this equation on a bounded domain, with homogeneous Neuman boundary conditions.

## Rock4 integration 

In [10]:
def plot_rock_sol():

    xmin = -2.
    xmax = 2.
    nx = 401
    x = np.linspace(xmin, xmax, nx)
    
    tini = 0. 
    tend = 0.5      
    
    hm = heat_model(xmin=xmin, xmax=xmax, nx=nx)
    fcn_rock  = hm.fcn_rock
    
    freq = np.pi/((xmax-xmin)/2)
    yini = np.cos(freq*x)
    #yini = np.sign(x)
    yexa = np.exp(-freq**2*(tend-tini))*yini
          
    tol = 1.e-6
    sol = integration.rock4(tini, tend, yini, fcn_rock, tol)

    yerr_exa = np.abs(yexa - sol.y)
    
    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, sol.y)
    fig_err = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Global error")
    fig_err.line(x, yerr_exa, legend_label="With respect to exact solution", color="green")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err))
    
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of computed steps : {sol.nstep:d}")
    print(f"Number of accepted steps : {sol.naccpt:d}")
    print(f"Number of rejected steps : {sol.nrejct:d}")
    print(f"Maximum number of stage used : {sol.nstage:d}")
    
plot_rock_sol()

Number of function evaluations : 1322
Number of computed steps : 32
Number of accepted steps : 32
Number of rejected steps : 0
Maximum number of stage used : 67


## RK45 integration 

In [9]:
def plot_RK45_sol():

    xmin = -2.
    xmax = 2.
    nx = 401
    x = np.linspace(xmin, xmax, nx)
    
    tini = 0. 
    tend = 0.5        
    
    hm = heat_model(xmin=xmin, xmax=xmax, nx=nx)
    fcn  = hm.fcn
    
    freq = np.pi/((xmax-xmin)/2)
    yini = np.cos(freq*x)
    
    yexa = np.exp(-freq**2*(tend-tini))*yini
          
    tol = 1.e-6
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    y = sol.y[:,-1]

    yerr_exa = np.abs(yexa - y)
    
    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, y)
    fig_err = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Global error")
    fig_err.line(x, yerr_exa, legend_label="With respect to exact solution", color="green")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err))
    
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of accepted steps : {(sol.t.size-1):d}")
    
plot_RK45_sol()

Number of function evaluations : 42134
Number of accepted steps : 6013


## Radau5 integration

In [17]:
def plot_radau_sol():

    xmin = -2.
    xmax = 2.
    nx = 401
    x = np.linspace(xmin, xmax, nx)
    
    tini = 0. 
    tend = 1.            
    
    hm = heat_model(xmin=xmin, xmax=xmax, nx=nx)
    fcn_radau  = hm.fcn_radau
                        
    freq = np.pi/((xmax-xmin)/2)
    yini = np.cos(freq*x)
    
    yexa = np.exp(-freq**2*(tend-tini))*yini
        
    tol = 1.e-6
    sol = integration.radau5(tini, tend, yini, fcn_radau, njac=1, rtol=tol, atol=tol, iout=0)
    
    yerr_exa = np.abs(yexa - sol.y)

    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, sol.y)
    fig_err = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Global error")
    fig_err.line(x, yerr_exa, legend_label="With respect to exact solution", color="green")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err))
    
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of jacobian evaluations : {sol.njev:d}")
    print(f"Number of computed steps : {sol.nstep:d}")
    print(f"Number of accepted steps : {sol.naccpt:d}")
    print(f"Number of rejected steps : {sol.nrejct:d}")
    print(f"Number of LU decompositions : {sol.ndec:d}")
    print(f"Number of forward-backward substitutions : {sol.nsol:d}")
    
plot_radau_sol()

Number of function evaluations : 69
Number of jacobian evaluations : 1
Number of computed steps : 15
Number of accepted steps : 15
Number of rejected steps : 0
Number of LU decompositions : 12
Number of forward-backward substitutions : 18


## Rock4 integration (evolution of time step)

In [16]:
def plot_rock_dt_sol():

    xmin = -2.
    xmax = 2.
    nx = 801
    x = np.linspace(xmin, xmax, nx)
    
    tini = 0. 
    tend = 0.5           
    
    hm = heat_model(xmin=xmin, xmax=xmax, nx=nx)
    fcn_rock  = hm.fcn_rock
    
    freq = np.pi/((xmax-xmin)/2)
    yini = np.cos(freq*x)
    #yini = 1.*(x>0) - 1.*(x<0)
        
    tol = 1.e-6   
    sol = integration.rock4(tini, tend, yini, fcn_rock, tol)
    soldt = integration.rock4_dt(tini, tend, yini, fcn_rock, tol)
    tsol =  soldt.tsol
    dt = tsol[1:] - tsol[:-1]
    dt_min = np.min(dt)
    dt_max = np.max(dt)


    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, sol.y)
    fig_dt = figure(plot_height=300, plot_width=900, x_range=(tini, tend), y_range=(0.1*dt_min, 10*dt_max), y_axis_type="log", title="Time step")
    fig_dt.quad(top=tsol[1:]-tsol[:-1], bottom=1.e-16, left=tsol[:-1], right=tsol[1:], fill_color="navy", line_color="white", alpha=0.5) 

    show(column(fig_sol, fig_dt))
    
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of computed steps : {sol.nstep:d}")
    print(f"Number of accepted steps : {sol.naccpt:d}")
    print(f"Number of rejected steps : {sol.nrejct:d}")
    print(f"Maximum number of stage used : {sol.nstage:d}")
    
plot_rock_dt_sol()

Number of function evaluations : 4807
Number of computed steps : 87
Number of accepted steps : 87
Number of rejected steps : 0
Maximum number of stage used : 67



## Back to the heat equation on $\mathbb{R}$

In a second part, we check how well the numerical solution on a bounded domain can approximate a solution of the heat equation on $\mathbb{R}$.


In [7]:
def plot_radau_sol():

    xmin = -2.
    xmax = 2.
    nx = 2001
    x = np.linspace(xmin, xmax, nx)
    
    tini = 1e-5
    tend = 0.01             
    
    hm = heat_model(xmin=xmin, xmax=xmax, nx=nx)
    fcn_radau  = hm.fcn_radau
                        
    yini = hm.fcn_exact(tini)
    
    sol_funda = hm.fcn_exact(tend)
    
    sol_quasi_exa = integration.radau5(tini, tend, yini, fcn_radau, njac=1, rtol=1.e-12, atol=1.e-12, iout=0)
        
    tol = 1.e-4
    sol = integration.radau5(tini, tend, yini, fcn_radau, njac=1, rtol=tol, atol=tol, iout=0)
    
    yerr_funda = np.abs(sol_funda - sol.y)
    yerr_quasi_exa = np.abs(sol_quasi_exa.y - sol.y)

    fig_sol = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution")
    fig_sol.x(x, sol.y)
    fig_err = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Global error")
    fig_err.x(x, yerr_funda, legend_label="With respect to the fundamental solution", color="green")  
    fig_err.x(x, yerr_quasi_exa, legend_label="With respect to the quasi exact solution", color="crimson")
    fig_err.legend.location = "top_left"


    show(column(fig_sol, fig_err))
    
    print(f"Number of function evaluations : {sol.nfev:d}")
    print(f"Number of jacobian evaluations : {sol.njev:d}")
    print(f"Number of computed steps : {sol.nstep:d}")
    print(f"Number of accepted steps : {sol.naccpt:d}")
    print(f"Number of rejected steps : {sol.nrejct:d}")
    print(f"Number of LU decompositions : {sol.ndec:d}")
    print(f"Number of forward-backward substitutions : {sol.nsol:d}")
    
plot_radau_sol()

Number of function evaluations : 111
Number of jacobian evaluations : 1
Number of computed steps : 24
Number of accepted steps : 24
Number of rejected steps : 0
Number of LU decompositions : 24
Number of forward-backward substitutions : 29
